# Assign WP population density weights to FB

In [14]:
import os
import sys
import argparse 
import numpy as np
import rasterio
import pandas as pd
import numpy as np
import tqdm
import itertools
import multiprocess as mp
import glob
import gdal
import subprocess

In [15]:
print('--------------------------------------------------------------------------------------')
print('Population data process 4/7: assign population density weights to FB settlement pixels')
print('--------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------
Population data process 4/7: assign population density weights to FB settlement pixels
--------------------------------------------------------------------------------------


In [16]:
#setting up paths for files
#top_folder="/home/dohyungkim/population"
#ISO="IDN"
#year="2015"

In [ ]:
# Initialize parser 
my_parser = argparse.ArgumentParser(description='initial input')
my_parser.add_argument('top_folder',metavar='top_folder',type=str,help='working folder')
my_parser.add_argument('ISO',metavar='ISO',type=str,help='3 character country iso code')
my_parser.add_argument('year',metavar='year',type=str,help='population year')
args = my_parser.parse_args()
top_folder = args.top_folder
ISO = args.ISO
year = args.year
if not os.path.isdir(top_folder):
    print('The path specified does not exist')
    sys.exit()

In [17]:
fb_path=os.path.join(top_folder,ISO,"fb_data")
fb_tile_path=os.path.join(top_folder,ISO,"fb_data","fb_tiles")
wp_path=os.path.join(top_folder,ISO,"wp_data")
out_tile_path=os.path.join(top_folder,ISO,"fb_data","out_tiles")

In [18]:
#fb_files=glob.glob(os.path.join(fb_tile_path,'*.tif'))
fb_files=os.listdir(fb_tile_path)
wp_file=os.path.join(wp_path,'prd_files',"rf_predict.vrt")

In [19]:
#open worldpop population file
wp_data = rasterio.open(wp_file)
wp_arr = wp_data.read(1)

In [7]:
wp_arr[np.where(wp_arr<0)]=0
wp_min = np.min(wp_arr[np.where(wp_arr>0)])

In [8]:
def get_val(fb_file):
    import time
    try:
        fb_data=rasterio.open(os.path.join(fb_tile_path,fb_file))
        fb_arr=fb_data.read(1)
        fb_where=np.array(np.where(~np.isnan(fb_arr)))
        fb_where=fb_where.T
        for n in range(len(fb_where)):
            i,j=fb_where[n]
            x, y  = fb_data.xy(i, j)
            val = wp_arr[wp_data.index(x, y)]*0.09
            if val!=0: #wp value can be 0 when fb value is not 0
                fb_arr[i,j]=val
            else:
                fb_arr[i,j]=wp_min
        
        with rasterio.open(
            os.path.join(out_tile_path,fb_file),
            'w',
            driver='GTiff',
            height=fb_arr.shape[0],
            width=fb_arr.shape[1],
            count=1,
            dtype=fb_arr.dtype,
            crs=fb_data.crs,
            transform=fb_data.transform
        ) as dst:
            dst.write(fb_arr, 1)
        
        return(1)
    except:
        return(0)

In [9]:
p = mp.Pool()

In [10]:
results = list(tqdm.tqdm(p.imap_unordered(get_val, fb_files), total=len(fb_files)))

100%|██████████| 4/4 [00:02<00:00,  1.78it/s]


In [11]:
p.close()
p.join()

In [12]:
os.chdir(out_tile_path)
out_files=glob.glob(out_tile_path + '/*.tif')
vrt=gdal.BuildVRT(os.path.join(out_tile_path,"fb_weight.vrt"), out_files)
vrt.FlushCache()